In [ ]:
import pandas as pd
import numpy as np
# import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

import riiideducation

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MultiLabelBinarizer
import gc
from keras.layers import Reshape
import gensim
import pickle
from keras import layers
import tensorflow as tf
import gc


In [ ]:
train_csv =  pd.read_csv("../input/riiid/filtered_user_df.csv")

In [ ]:

with open(r"../input/riiid-results/embedding_matrix.pkl", "rb") as input_file:
    embedding_matrix = pickle.load(input_file) 

In [ ]:
embedding_matrix.shape

# Preprocess

In [ ]:
train_csv = train_csv[train_csv['answered_correctly'] != -1].reset_index(drop=True)

In [ ]:
train_csv['lag'] = train_csv.groupby('user_id')['answered_correctly'].shift()
cum = train_csv.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
train_csv['user_correctness'] = cum['cumsum'] / cum['cumcount']
train_csv.drop(columns=['lag'], inplace=True)

In [ ]:
user_agg = train_csv.groupby('user_id')['answered_correctly'].agg(['sum', 'count'])
content_agg = train_csv.groupby('content_id')['answered_correctly'].agg(['sum', 'count'])

In [ ]:
train_csv = train_csv.groupby('user_id').tail(24).reset_index(drop=True)

In [ ]:
train_csv['content_count'] = train_csv['content_id'].map(content_agg['count']).astype('int32')
train_csv['content_id_correctness'] = train_csv['content_id'].map(content_agg['sum'] / content_agg['count'])

In [ ]:
train_csv['prior_question_had_explanation'].fillna(False, inplace=True)
train_csv['prior_question_elapsed_time'].fillna(0,inplace=True)

In [ ]:
lencoder = LabelEncoder()
train_csv['prior_question_had_explanation_enc'] = lencoder.fit_transform(train_csv['prior_question_had_explanation'])

In [ ]:
train_csv[:10]

In [ ]:
scaler = StandardScaler()
train_csv['prior_question_elapsed_time']=scaler.fit_transform(train_csv['prior_question_elapsed_time'].values.reshape(-1,1))

In [ ]:
train_csv[:10]

In [ ]:
embedding_matrix[45][0]

In [ ]:
embedding_matrix[185]

In [ ]:
len(embedding_matrix)

In [ ]:
len(train_csv)

In [ ]:
def func(content,pos):
    return embedding_matrix[content][pos]
    
    

In [ ]:
i = 0
while(i<20):
    train_csv['embed'+str(i)] = train_csv['content_id']
    train_csv['embed'+str(i)] = train_csv['embed'+str(i)].apply(lambda x : func(x,i))
    i = i+1

In [ ]:
del(embedding_matrix)

In [ ]:
i = 0
while(i<20):
    train_csv['lag'] = train_csv.groupby('user_id')['embed'+str(i)].shift()
    cum = train_csv.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
    train_csv['embed_avg'+str(i)] = np.average([1, 2, 3], weights=[1, 0, 1])
    train_csv.drop(columns=['lag'], inplace=True)
    i = i+1
    

In [ ]:

i = 0
while(i<20):
    train_csv['embed_avg'+str(i)][0] = 0  #to ask - how to fill first entry which has no history
    i = i+1

In [ ]:
#embed_avg - history avg of q embedding
#embed - cuurent q embedding 

In [ ]:
train_csv.head()

In [ ]:
valid_csv = train_csv.groupby('user_id').tail(6)
train_csv.drop(valid_csv.index, inplace=True)

In [ ]:
train_csv[:5]

In [ ]:
valid_csv[:5]

In [ ]:
train_csv.columns

In [ ]:
# train_csv.drop(columns = 'embed0_avg',inplace = True)

In [ ]:
train_csv.columns

In [ ]:
gc.collect()

# Train

In [ ]:
features = [
    'prior_question_elapsed_time',
    'prior_question_had_explanation_enc',
    'user_correctness',
    'content_count',
    'content_id_correctness',
    'embed0', 'embed1', 'embed2',
    'embed3', 'embed4', 'embed5', 
    'embed6', 'embed7', 'embed8', 
    'embed9','embed10', 'embed11',
    'embed12', 'embed13', 'embed14',
    'embed15','embed16', 'embed17',
    'embed18', 'embed19',
    'embed_avg0',
    'embed_avg1', 'embed_avg2', 'embed_avg3', 'embed_avg4', 'embed_avg5',
    'embed_avg6', 'embed_avg7', 'embed_avg8', 'embed_avg9', 'embed_avg10',
    'embed_avg11', 'embed_avg12', 'embed_avg13', 'embed_avg14',
    'embed_avg15', 'embed_avg16', 'embed_avg17', 'embed_avg18',
    'embed_avg19'
     ]
target = 'answered_correctly'

params = {
    'objective': 'binary',
    'seed': 42,
    'metric': 'auc',
    'device' : 'gpu',
    'learning_rate': 0.05,
    'max_bin': 100,
    'num_leaves': 80
}

In [ ]:
tr_data = lgb.Dataset(train_csv[features], label=train_csv[target])
va_data = lgb.Dataset(valid_csv[features], label=valid_csv[target])

model = lgb.train(
    params, 
    tr_data, 
    num_boost_round=10000,
    valid_sets=[tr_data, va_data], 
    early_stopping_rounds=10,
    verbose_eval=50
)

# model.save_model(f'model.txt')
lgb.plot_importance(model, importance_type='gain')
plt.show()